<a href="https://colab.research.google.com/github/udlbook/iclimbtrees/blob/main/notebooks/SAT2/DPLL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The DPLL algorithm

The purpose of this Python notebook is to implement the DPLL algorithm.

Work through the cells below, running each cell in turn. In various places you will see the words "TODO". Follow the instructions at these places and write code to complete the functions.

You can save a local copy of this notebook in your Google account and work through it in Colab (recommended) or you can download the notebook and run it locally using Jupyter notebook or similar. If you are using CoLab, we recommend that turn off AI autocomplete (under cog icon in top-right corner), which will give you the answers and defeat the purpose of the exercise.

A fully working version of this notebook with the complete answers can be found [here](https://colab.research.google.com/github/udlbook/iclimbtrees/blob/main/notebooks/SAT2/DPLL_Answers.ipynb).

Contact me at iclimbtreesmail@gmail.com if you find any mistakes or have any suggestions.

In [ ]:
# Used to take a deep copy of list of clauses
import copy

This is the main unit propagation routine.  It only does something if there is at least one unit clause.

As argument, it takes a set of clauses representing a CNF formula. There are represented as a list of sets containing the indices of the variable, with minus signs denoting the complement.  For example, the formula

$$ \phi = (x_1 \lor x_2)\land (\overline{x}_1\lor x_3)$$

would be represented as [{1, 2}, {-1, 3}].

The routine returns False if a contradiction is found (the formula is UNSAT), or retuns the simplified set of clauses if propagation completes.  If this set is empty, then the formula is SAT. If not, then there is more work to do.

In [ ]:
def unit_propagation(current_clauses_in):
    # Take a deep copy of the current clauses
    current_clauses = copy.deepcopy(current_clauses_in)

    while True:
        print(f"Current Clauses: {current_clauses}")
        # Find all unit clauses in the current set of clauses
        unit_clauses = [clause for clause in current_clauses if len(clause) == 1]

        if not unit_clauses:
            # No more unit clauses found, propagation stops.
            print("--- Unit Propagation Halted ---")
            print(f"No more unit clauses to resolve.")
            return current_clauses

        # Take the first unit literal found (e.g., {1} gives literal 1)
        unit_literal = list(unit_clauses[0])[0]
        complement_literal = -unit_literal

        print(f"Conditioning on literal: {unit_literal}")

        # This list will contain the clauses after the unit resolution
        new_clauses = []
        contradiction = False

        #  Run through each clause in turn
        for clause in current_clauses:
            #1. If this clause is a unit clause containing the complement literal
            # then we have $xi and not(x_i)$,  so return FALSE
            if clause == {complement_literal}:
              return False

            # 2. Check if this clause is satisfied by the unit literal.
            # If so it is satisfied so do nothing
            if unit_literal in clause:
                # This clause is satisfied, so we discard it.
                continue

            # 3. Check if the clause contains the complement of the unit literal
            # If so then remove the complement from this clause
            # (you can use https://www.w3schools.com/python/ref_set_remove.asp)
            elif complement_literal in clause:
                # The complement is in the clause. The complement must be false,
                # so we remove it from the clause, and append this modified
                # clause to new_clauses
                clause.remove(complement_literal)
                new_clauses.append(clause)
            # 4.  If none of these cases are true, then we the current clause
            # Does not contain this literal, so we  we just
            # append it to new_clauses
            else:
                new_clauses.append(clause)

        current_clauses = new_clauses

The DPLL algorithm first tries to perform unit resolution to simplify the formula.  This may result in either a SAT or UNSAT result, or may leave other clauses, which can be conditioned on.   

In this context, we can "condition" on a literal $x_i$ by simply adding it as a unit clause, and the next round of unit resolution simplifies the formula accordingly.  In other words, we never need to explicitly write out the logic by which the conditioning operation simplifies the formula.

In [ ]:
def dpll(clauses):
    """
    Checks for satisfiability of a formula unit propagation
    and recursive conditioniong

    Args:
        clauses: A list of sets, where each set represents a clause
                 (e.g., [{1, 2}, {-1, 3}]).

    Returns:
        True if the formula is satisfiable, False otherwise.
    """
    print(f"\nAttempting to solve with clauses: {clauses}")

    # Step 1: Apply unit propagation
    # TODO -- call the routine above
    # Replace this line
    propagated_clauses = copy.deepcopy(clauses)

    # Step 2: Check base cases after propagation
    if propagated_clauses is False:
        print("  -> Contradiction found during unit propagation. UNSAT.")
        return False  # Contradiction found, formula is unsatisfiable
    if not propagated_clauses:
        print("  -> All clauses satisfied after unit propagation. SAT.")
        return True  # All clauses satisfied, formula is satisfiable

    # Step 3: If unit propagation halted and clauses remain, pick a variable to branch on
    # Pick an arbitrary variable to condition on -- we'll choose the one with the lowest index, to make this easier
    variable_to_condition = min(abs(literal) for current_clause in propagated_clauses for literal in current_clause)
    print(f"  -> Unit propagation halted. Conditioning on variable: {variable_to_condition}")


    # Step 4: Recursive calls

    # Try assigning to true
    print(f"    -> Trying {variable_to_condition} = False")
    # TODO, add {-variable_to_condition} as a unit clause
    # Call dpll recursively with this new list of clauses
    # If dpll returns true, then return true, otherwise keep going
    # Replace this line:
    return True

    # If the above branch returned False, try assigning the variable to
    # Try assigning the variable to True (add {variable_to_condition} as a unit clause)
    print(f"    -> Trying {variable_to_condition} = True")
    # TODO, add {-variable_to_condition} as a unit clause
    # Call dpll recursively with this new list of clauses
    # If dpll returns true, then return true, otherwise keep going
    # Replace this line:
    return True

    # If neither assignment leads to a satisfiable solution, the current path is unsatisfiable
    print(f"  -> Both branches for variable {variable_to_condition} led to UNSAT. Backtracking.")
    print("======================================================================================")
    return False

Let's try with the example from the main text which we saw to be SAT

In [7]:
formula_1 = [{1, 2}, {1, -2, -3}, {-2, 5}, {3, 6, 8}, {4, 6, 7}, {-6, 7}, {2, -4, 6}, {1, -4, 6}, {3, -6, 7}, {-2, -7, 8}, {3, -7, -8},
             {-1, 2, 3}, {2, -3, 4}, {-3,-4,5}, {-3, -5, -7}, {-1, 2, -4}, {-1, 4, 6}, {3, -4, -6}]
dpll(formula_1)


Attempting to solve with clauses: [{1, 2}, {1, -3, -2}, {5, -2}, {8, 3, 6}, {4, 6, 7}, {-6, 7}, {2, -4, 6}, {1, -4, 6}, {-6, 3, 7}, {8, -7, -2}, {-8, -7, 3}, {2, 3, -1}, {2, 4, -3}, {-3, -4, 5}, {-7, -5, -3}, {2, -4, -1}, {4, -1, 6}, {-6, 3, -4}]
  -> Unit propagation halted. Conditioning on variable: 1
    -> Trying 1 = False


True

If you have implemeted this correctly, it will follow exactly the same sequence of operations as the example in the multi-panel figure.